In [ ]:
# ============================================================================
# EXEMPLO 1: OTIMIZAÇÃO DE HIPERPARÂMETROS PARA DETECÇÃO DE FRAUDES
# Modelagem Matemática Aplicada à Otimização em Análise de Dados
# Prof. Maurizio Prizzi - CEUB
# ============================================================================

"""
OBJETIVO: Demonstrar como otimizar hiperparâmetros de modelos de Machine Learning
para maximizar a detecção de fraudes financeiras minimizando falsos positivos.

CONCEITOS MATEMÁTICOS:
- Otimização multi-objetivo
- Busca em grade (Grid Search) ou busca aleatória (Randomized Search)
- Validação cruzada estratificada
- Função de fitness customizada
- Análise de trade-offs

VERSÃO REVISADA: Inclui correções de erros, melhorias de robustez e eficiência.
"""

# Importando bibliotecas necessárias
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV, StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score
from sklearn.metrics import precision_score, recall_score, f1_score, roc_curve
from sklearn.datasets import make_classification
import matplotlib.pyplot as plt
import seaborn as sns
import joblib
import time
import warnings
warnings.filterwarnings('ignore')

print("=" * 80)
print("EXEMPLO 1: OTIMIZAÇÃO DE HIPERPARÂMETROS PARA DETECÇÃO DE FRAUDES")
print("=" * 80)

# ============================================================================
# 1. GERAÇÃO DE DATASET SINTÉTICO DE TRANSAÇÕES FINANCEIRAS
# ============================================================================

def criar_dataset_transacoes_financeiras():
    """
    Cria um dataset sintético que simula transações bancárias reais
    com características típicas de detecção de fraude
    """
    print("\n📊 Gerando dataset sintético de transações bancárias...")
    
    # Parâmetros para simular dados realistas de transações
    n_transacoes = 15000  # 15 mil transações
    n_features = 18       # 18 características por transação
    n_informativas = 14   # 14 features realmente úteis
    
    # Distribuição desbalanceada típica: 96% legítimas, 4% fraudulentas
    weights = [0.96, 0.04]
    
    # Gerando dados com padrões complexos
    X, y = make_classification(
        n_samples=n_transacoes,
        n_features=n_features,
        n_informative=n_informativas,
        n_redundant=2,
        n_clusters_per_class=2,  # Múltiplos padrões de fraude
        weights=weights,
        flip_y=0.02,  # 2% de ruído nos labels
        random_state=42
    )
    
    # Nomes das features baseados em dados bancários reais
    feature_names = [
        'valor_transacao_log',     # Log do valor da transação
        'hora_do_dia',             # Hora da transação (0-23)
        'dia_da_semana',           # Dia da semana (1-7)
        'freq_transacoes_hora',    # Frequência de transações na última hora
        'valor_medio_30d',         # Valor médio das transações nos últimos 30 dias
        'desvio_padrao_valores',   # Variabilidade histórica dos valores
        'tempo_ultima_transacao',  # Minutos desde a última transação
        'tentativas_login_24h',    # Tentativas de login nas últimas 24h
        'localizacao_habitual',    # Se a localização é habitual (0-1)
        'dispositivo_conhecido',   # Se o dispositivo é conhecido (0-1)
        'idade_conta_meses',       # Idade da conta em meses
        'saldo_medio_30d',        # Saldo médio dos últimos 30 dias
        'limite_credito_utilizado',  # % do limite de crédito utilizado
        'score_comportamental',    # Score do contexto (0-100)
        'num_cartoes_ativos',      # Número de cartões ativos
        'transacoes_int_30d',      # Transações internacionais nos últimos 30 dias
        'velocidade_digitacao',    # Velocidade de digitação normalizada
        'padrão_normalizado'         # Score de padrão normalizado
    ]
    
    # Criando DataFrame
    df = pd.DataFrame(X, columns=feature_names)
    df['e_fraude'] = y  # 0 = legítima, 1 = fraude
    
    # Forçando features categóricas a serem binárias
    df['localizacao_habitual'] = (df['localizacao_habitual'] > 0).astype(int)
    df['dispositivo_conhecido'] = (df['dispositivo_conhecido'] > 0).astype(int)
    
    # Verificação da proporção de fraudes
    fraude_ratio = df['e_fraude'].mean()
    if not (0.03 <= fraude_ratio <= 0.05):
        print(f"⚠️ Aviso: Proporção de fraudes ({fraude_ratio:.2f}) fora do intervalo esperado (0.03-0.05).")
    
    # Validação de features categóricas
    for col in ['localizacao_habitual', 'dispositivo_conhecido']:
        if not df[col].isin([0, 1]).all():
            print(f"⚠️ Erro: A coluna {col} contém valores não binários.")
            raise ValueError(f"A coluna {col} deve conter apenas valores 0 ou 1.")
    
    # Estatísticas do dataset
    total = len(df)
    fraudulentas = len(df[df['e_fraude'] == 1])
    legitimas = len(df[df['e_fraude'] == 0])
    
    print(f"✅ Dataset criado com sucesso!")
    print(f"   📈 Total de transações: {total:,}")
    print(f"   ✅ Transações legítimas: {legitimas:,} ({legitimas/total*100:.2f}%)")
    print(f"   🚨 Transações fraudulentas: {fraudulentas:,} ({fraudulentas/total*100:.2f}%)")
    
    return df

# Criando o dataset
df_transacoes = criar_dataset_transacoes_financeiras()

# ============================================================================
# 2. FUNÇÃO OBJETIVO MULTI-CRITERIO PARA OTIMIZAÇÃO
# ============================================================================

def funcao_objetivo_fintech(y_true, y_pred, peso_deteccao=0.5, peso_precisao=0.3, peso_experiencia=0.2):
    """
    Função objetivo que simula os critérios de negócio de uma fintech
    
    Parâmetros:
    - y_true: Labels verdadeiros
    - y_pred: Predições do modelo
    - peso_deteccao: Peso para recall (detectar fraudes)
    - peso_precisao: Peso para precisão (evitar alarmes falsos)
    - peso_experiencia: Peso para experiência do usuário (baixo FPR)
    
    Retorna: Score que queremos MAXIMIZAR
    """
    # Validação dos pesos
    if not (0 <= peso_deteccao <= 1 and 0 <= peso_precisao <= 1 and 0 <= peso_experiencia <= 1):
        print("⚠️ Erro: Os pesos devem estar entre 0 e 1.")
        raise ValueError("Os pesos devem estar entre 0 e 1.")
    if abs(peso_deteccao + peso_precisao + peso_experiencia - 1.0) > 1e-10:
        print("⚠️ Erro: Os pesos devem somar 1.")
        raise ValueError("Os pesos devem somar 1.")
    
    # Calculando métricas fundamentais
    recall = recall_score(y_true, y_pred, zero_division=0)
    precision = precision_score(y_true, y_pred, zero_division=0)
    
    # Taxa de falsos positivos (impacta experiência do usuário)
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
    fpr = fp / (fp + tn) if (fp + tn) > 0 else 0
    
    # Alertas para casos extremos
    if (fp + tn) == 0:
        print("⚠️ Aviso: Nenhuma transação negativa detectada. FPR ajustado para 0.")
    if sum(y_pred) == 0:
        print("⚠️ Aviso: Nenhuma previsão positiva feita. Precision ajustada para 0.")
    
    # Score de experiência do usuário (1 - FPR)
    experiencia_usuario = 1 - fpr
    
    # Combinação linear ponderada dos objetivos
    score_final = (peso_deteccao * recall + 
                   peso_precisao * precision + 
                   peso_experiencia * experiencia_usuario)
    
    return score_final

# Função de scoring para GridSearchCV/RandomizedSearchCV
def score_customizado_fintech(estimator, X, y):
    """Wrapper da função objetivo para uso no GridSearchCV/RandomizedSearchCV"""
    y_pred = estimator.predict(X)
    return funcao_objetivo_fintech(y_true=y, y_pred=y_pred)

# ============================================================================
# 3. PREPARAÇÃO E DIVISÃO DOS DADOS
# ============================================================================

print("\n=== Preparando os dados para modelagem ===")

# Separando features (X) e target (y)
X = df_transacoes.drop('e_fraude', axis=1)
y = df_transacoes['e_fraude']

# Divisão estratificada para manter proporção de fraudes
X_train, X_test, y_train, y_test = train_test_split(
    X, y, 
    test_size=0.25,      # 25% para teste
    random_state=42, 
    strategy=y           # Estratificação
)

# Normalização apenas para features numéricas
colunas_numericas = [col for col in X.columns if col not in ['localizacao_habitual', 'dispositivo_conhecido']]
scaler = StandardScaler()
X_train_norm = X_train.copy()
X_test_norm = X_test.copy()
X_train_norm[colunas_numericas] = scaler.fit_transform(X_train[colunas_numericas])
X_test_norm[colunas_numericas] = scaler.transform(X_test[colunas_numericas])

print(f"\n✅ Dados preparados com sucesso:")
print(f"Conjunto de treinamento: {X_train.shape[0]:,} transações")
print(f"Conjunto de teste: {X_test.shape[0]:,} transações")
print(f"Features por transação: {X_train.shape[1]}")

# ============================================================================
# 4. DEFINIÇÃO DO ESPAÇO DE BUSCA PARA OTIMIZAÇÃO
# ===========================================================================

print("\n=== Definindo o espaço de busca para otimização ===")

# Modelo base: Random Forest com peso de classe
modelo_rf = RandomForestClassifier(random_state=42, n_jobs=-1, class_weight='balanced')

# Grid de hiperparâmetros
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [10, 15, None],  # Ajustado para incluir profundidade menor
    'criterion': ['gini', 'entropy'],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 3, 5],
    'max_features': ['sqrt', 'log2', max(1, int(0.6 * X.shape[1]))]  # Correção
}

# Flag para escolher entre GridSearchCV e RandomizedSearchCV
use_random_search = True

total_combinacoes = 1 if use_random_search else np.prod([len(v) for v in param_grid.values()])
if use_random_search:
    total_combinacoes = 50  # Número de iterações para RandomizedSearchCV

print(f"\n✅ Espaço de busca definido:")
print(f"Total de combinações' if not use_random_search else 'iterações': {total_combinacoes:,}")
print(f"Estimativa de tempo: ~{total_combinacoes * 0.2:.0f} segundos")

# ===========================================================================
# 5. EXECUÇÃO DA OTIMIZAÇÃO
# ===========================================================================

print("\n==== Iniciando a otimização de hiperparâmetros ===")

# Configurando o SearchCV
if use_random_search:
    searcher_cv = RandomizedSearchCV(
        estimator=modelo_rf,
        param_distributions=param_grid,
        n_iter=50,
        scoring=scorer_customizado_fintech_score,
        cv=StratifiedKFold(n_splits=5, shuffle=True, random_state=42),
        n_jobs=-1,
        verbose=1,
        random_state=42,
        return_train_score=True
    )
else:
    searcher_cv = GridSearchCV(
        estimator=modelo_rf_rf,
        param_grid=param_grid_grid,
        scoring=scorer_customizado_fintech_score,
        cv=StratifiedKFold(n_splits=5, shuffle=True, random_state=42),
        n_jobs=-1,
        verbose=1,
        return_train_score=True
    )

# Executando a busca com medição de tempo
print("Executando busca (pode levar alguns minutos)...")
start_time = time.time()
searcher_cv.fit(X_train_norm, y_train)
elapsed_time = time.time() - start_time

print(f"\n✅ Otimização concluída em {elapsed_time:.2f} segundos!")
print(f"Melhor score (CV): {searcher_cv.best_score_:.4f}")

print(f"\nMelhor hiperparâmetros encontrados:")
for param, value in searcher_cv.best_params_.items():
    print(f"  {param}: {value}")

# Salvando modelo, scaler e resultados
joblib.dump(searcher_cv.best_estimator_, 'modelo_fraudes_rf.pkl')
joblib.dump(scaler, 'scaler_fraudes.pkl')
resultados_cv = pd.DataFrame(searcher_cv.cv_results_)
resultados_cv.to_csv('resultados_search_cv.csv', index=False)
print("\n💾 Modelo, scaler e resultados salvos como 'modelo_fraudes_rf.pkl', 'scaler_fraudes.pkl' e 'resultados_search_cv.csv'")

# ===========================================================================
# 6. AVALIAÇÃO DO MODELO OTIMIZADO
# ===========================================================================

print("\n=== Avaliando a performance do modelo otimizado ===")

# Modelo com melhores hiperparâmetros
modelo_optimizado = searcher_cv.best_estimator_

# Predições no conjunto de teste
y_pred_test = modelo_optimizado.predict(X_test_norm)
y_proba_test = modelo_optimizado.predict_proba(X_test_norm)[:, 1]

# Métricas detalhadas
recall_final = recall_score(y_test, y_pred_test)
precision_final = precision_score(y_test, y_pred_test)
f1_final = f1_score(y_test, y_pred_test)
auc_final = roc_auc_score(y_test, y_proba_test)

print(f"\n📈 Métricas de Performance Final:")
print(f"Recall (Detecção de Fraudes): {recall_final:.4f}")
print(f"Precisão (Qualidade): {precision_final:.4f}")
print(f"F1-Score: {f1_final:.4f}")
print(f"AUC-ROC: {auc_final:.4f}")

# Relatório detalhado de classificação
print("\nRelatório Detalhado de Classificação:")
print(classification_report(y_true=y_test, y_pred=y_pred_test, 
                          target_names=['Legítima', 'Fraude'], 
                          digits=4))

# ===========================================================================
# 7. VISUALIZAÇÕES ESPECÍFICAS
# ===========================================================================

# Configuração global para plots
plt.style.use('default')
plt.rcParams['figure.figsize'] = [12, 12]
plt.rcParams['font.size'] = 10

# Criando subplots
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2)

# Plot 1: Matriz de Confusão
cm = confusion_matrix(y_test, y_pred_test)
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', ax=ax1,
            xticklabels=['Legítima', 'Fraude'],
            yticklabels=['Legítima', 'Fraude'])
ax1.set_title('Predição Matriz de Confusão', fontsize=12, fontweight='bold')
ax1.set_xlabel('Predição')
ax1.set_ylabel('Real')

# Plot 2: Importância das Features
feature_importance_df = pd.DataFrame({
    'feature': X.columns,
    'score': modelo_optmodelo_score.importances_
}).sort_values('score', ascending=False).head(10)

ax2.barh(range(len(feature_importance_df))),
feature_importance_df['score'],
)
ax2.set_yticks(range(len(feature_importance_df)))
ax2.set_yticklabels(feature_importance_df['feature'])
ax2.set_title('Top 10 Features Mais Importantes')
ax2.set_xlabel('Importância')
ax2.grid(axis='x', alpha=0.2)

# Plot 3: Curva ROC
fpr, tpr, _ = roc_curve(y_test, y_proba_test)
ax3.plot(fpr, tpr, color='blue', linewidth=2, label=f'ROC (AUC={auc_final:.4f})')
ax3.plot([0, 1], [0, 1], 'r--', alpha=0.5, label='Random')
ax3.set_xlabel('Taxa de Falsos Positivos')
ax3.set_ylabel('Taxa de Verdadeiros Positivos')
ax3.set_title('Curva ROC')
ax3.legend()
ax3.grid(alpha=0.2)

# Plot 4: Análise de Convergência (dispersão)
scores_mean = resultados_cv['mean_test_score']
ax4.scatter(range(len(scores_mean)), scores_mean, c=scores_mean, cmap='viridis', alpha=0.7)
ax4.axhline(y=searcher_cv.best_score_, color='red', linestyle='--', 
            label=f'Melhor Score: {searcher_cv.best_score_:.4f}')
ax4.set_xlabel('Configuração de Hiperparâmetros')
ax4.set_ylabel('Score Cross-Validation')
ax4.set_title('Convergência da Otimização')
ax4.legend()
ax4.grid(alpha=0.2)

plt.tight_layout()
plt.show()

# ===========================================================================
# 8. ANÁLISE DE TRADE-OFFS DE NEGÓCIO
# ===========================================================================

print("\n=== Análise de Trade-offs de Negócio ===")

# Calculando impacto financeiro hipotético
total_transacoes_teste = len(y_test)
fraudes_reais = sum(y_test)
legitimas_reais = total_transacoes_teste - fraudes_reais

# Valores médios hipotéticos por transação
valor_medio_fraude = 850.0     # R$ 850 por fraude
custo_falso_positivo = 12.0    # R$ 12 de custo operacional por FP

# Calculando custos
tn, fp, fn, tp = cm.ravel()

# Benefícios: fraudes detectadas
beneficio_deteccao = tp * valor_medio_fraude

# Custos: fraudes não detectadas + falsos positivos
custo_fraudes_perdidas = fn * valor_medio_fraude
custo_falsos_positivos = fp * custo_falso_positivo
custo_total = custo_fraudes_perdidas + custo_falsos_positivos

# ROI do sistema
if custo_total > 0:
    roi = (beneficio_deteccao - custo_total) / custo_total * 100
else:
    roi = 0 if beneficio_deteccao == 0 else float('inf')

print(f"\nImpacto Financeiro Estimado (no conjunto de teste):")
print(f"Benefício - Fraudes detectadas: R$ {beneficio_deteccao:,.2f}")
print(f"Custo - Fraudes não detectadas: R$ {custo_fraudes_perdidas:,.2f}")
print(f"Custo - Falsos positivos: R$ {custo_falsos_positivos:,.2f}")
print(f"ROI do Sistema: {roi:.1f}%{' (infinito)' if roi == float('inf') else ''}")

# ===========================================================================
# 9. INSIGHTS E CONCLUSÕES
# ===========================================================================

print(f"\n{'='*80}")
print("INSIGHTS E CONCLUSÕES - EXEMPLO 1")
print(f"{'='*80}")

print(f"""
RESULTADOS DA OTIMIZAÇÃO:

✅ Performance Alcançada:
   • Recall (Detecção): {recall_final:.1%} das fraudes identificadas
   • Precision (Qualidade): {precision_final:.1%} das detecções são corretas  
   • F1-Score Balanceado: {f1_final:.4f}
   • AUC-ROC: {auc_final:.4f} (excelente discriminação)

🔧 Hiperparâmetros Otimizados:
   • Número de árvores: {searcher_cv.best_params_['n_estimators']}
   • Profundidade máxima: {searcher_cv.best_params_['max_depth']}
   • Critério de divisão: {searcher_cv.best_params_['criterion']}

💡 APLICAÇÕES PRÁTICAS:

1. Otimização Contínua:
   • Re-execução automática da busca com novos dados
   • Adaptação a mudanças nos padrões de fraude
   • Monitoramento de performance em produção

2. Balanceamento de Objetivos:
   • Trade-off entre detecção e experiência do usuário
   • Minimização de custos operacionais
   • Maximização do ROI do sistema antifraude

3. Escalabilidade:
   • Paralelização da busca de hiperparâmetros
   • Otimização distribuída para grandes volumes
   • AutoML para ajuste automático

🔬 CONCEITOS MATEMÁTICOS DEMONSTRADOS:

• Otimização combinatorial discreta (Grid/Random Search)
• Função objetivo multi-critério com pesos ajustáveis
• Validação cruzada estratificada
• Análise de importância por impureza de Gini
• Trade-off bias-variância em ensemble methods
""")

print(f"\n📚 Próximo: Exemplo 2 - Otimização Contínua de Threshold")
print(f"🔗 Repositório: https://github.com/maurizioprizzi/otimizacao-aplicada-ciencia-dados")
print(f"{'='*80}")

EXEMPLO 1: OTIMIZAÇÃO DE HIPERPARÂMETROS PARA DETECÇÃO DE FRAUDES
Técnicas inspiradas na Feedzai e outras fintechs líderes

📊 Gerando dataset sintético de transações bancárias...
⚠️ Aviso: Proporção de fraudes (0.051) fora do intervalo esperado (0.03-0.05).
✅ Dataset criado com sucesso!
   📈 Total de transações: 15,000
   ✅ Transações legítimas: 14,242 (94.9%)
   🚨 Transações fraudulentas: 758 (5.1%)

🔧 Preparando dados para modelagem...
✅ Dados preparados:
   📚 Conjunto de treino: 11,250 transações
   🧪 Conjunto de teste: 3,750 transações
   🔢 Features por transação: 18

🎯 Definindo espaço de busca para otimização de hiperparâmetros...
✅ Espaço de busca definido:
   🔍 Total de combinações: 486
   ⏱️ Estimativa de tempo: ~49 segundos

🚀 Iniciando otimização de hiperparâmetros...
   ⚡ Executando busca (pode levar alguns minutos)...
Fitting 5 folds for each of 486 candidates, totalling 2430 fits


KeyboardInterrupt: 